In [1]:
import sys
sys.path.append("..")
sys.path.append("my_library")
from my_library.config import *
from my_library.metrics.cmp_metrics import *
sys.path.append("..")
import os

import os


/home/mmd/anaconda3/envs/sfs2/lib/python3.9/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/mmd/anaconda3/envs/sfs2/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


this is the current directory:  /media/mmd/Samsung_T5/GitHub/UMD/metriche
This is the thesis folder, where all the code is located:  GitHub/UMD
Questo è samsung:/media/mmd/Samsung_T5


In [ ]:
'''ArithmeticErrormodels = ["br3", "adsq", "pdl1-3"] 
outcomes = ["BEST_RESPONSE_3", "HISTOLOGY", "PDL1_CATHEGORY"]
eval = "eval_1_single_fold"'''
'''
base_folder = os.path.join(working_directory,"eval")
subfolder = [name for name in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, name))]

results = []

eval = "eval"

models = ["br3", "adsq", "pdl1-3"] 
outcomes = ["BEST_RESPONSE_3", "HISTOLOGY", "PDL1_CATHEGORY"]

for m,o in zip(models, outcomes):
    print(m)
    print(o)

    outcome = o
    model = m
    root_dir = os.path.join(base_folder, m)'''

br3
BEST_RESPONSE_3
adsq
HISTOLOGY
pdl1-3
PDL1_CATHEGORY


In [3]:
import pandas as pd
csv_df = pd.read_csv(working_directory+"/projects/I3lung-sqadqc-project/annotations.csv")


# setup check

In [4]:
csv_df

,slide,EXT,FULL_PATH,CENTER,patient,FULL_I3LUNG_NAME,NUM_SLIDES,STUDY_TYPE,SLIDE_TYPE,COHORT,...,SITE_METS_IO_START,IO_START,IO_END,PROGRESSION_DATE,Z03_DATE,DEATH_EVENT_OC,PROGRESSION_EVENT_OC,THERAPY_END_EVENT_OC,I30_KRAS,I55_P53
0,DIG_PAT_1696327666,svs,D:/Digital_path/DIG_PAT_1696327666.svs,INT,INT1010133,INT1010133.svs,1,Retrospective,h/e,2,...,"lung,lymph nodes,bone",2022-07-04,2023-01-27,2023-02-13,2023-04-13,1.0,1.0,1.0,0.0,1.0
1,DIG_PAT_1696327717,svs,D:/Digital_path/DIG_PAT_1696327717.svs,INT,INT1010140,INT1010140.svs,1,Retrospective,h/e,1,...,NaN,2022-08-09,2023-07-14,2024-10-09,2024-10-09,0.0,0.0,1.0,NaN,NaN
2,DIG_PAT_1696325152,svs,D:/Digital_path/DIG_PAT_1696325152.svs,INT,INT1010001,INT1010001.svs,1,Retrospective,h/e,2,...,bone,2021-06-14,2023-09-08,2023-09-08,2023-09-08,0.0,0.0,0.0,1.0,NaN
3,DIG_PAT_1696325185,svs,D:/Digital_path/DIG_PAT_1696325185.svs,INT,INT1010002,INT1010002.svs,1,Retrospective,h/e,3,...,lung,2021-07-21,2023-08-17,2023-08-17,2023-08-17,0.0,0.0,0.0,NaN,NaN
4,DIG_PAT_1696325261,svs,D:/Digital_path/DIG_PAT_1696325261.svs,INT,INT1010003,INT1010003.svs,1,Retrospective,h/e,1,...,NaN,2022-07-13,2023-06-06,2024-10-02,2024-10-02,0.0,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,DIG_PAT_1728839427,tif,D:/Digital_path/DIG_PAT_1728839427.tif,MH,MH1040002,MH1040002_HE (2).tif,1,Prospective,h/e,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1038,DIG_PAT_1728843584,tif,D:/Digital_path/DIG_PAT_1728843584.tif,MH,MH1040073,MH1040073_HE (2).tif,1,Retrospective,h/e,3,...,"lung,lymph nodes,adrenal",2020-01-22,2021-03-22,2021-08-16,2023-01-25,0.0,1.0,1.0,0.0,0.0
1039,DIG_PAT_1728854618,tif,D:/Digital_path/DIG_PAT_1728854618.tif,MH,MH1040093,MH1040093_HE.tif,1,Prospective,h/e,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1040,DIG_PAT_1728911989,tif,D:/Digital_path/DIG_PAT_1728911989.tif,MH,MH1040120,MH1040120_HE (1).tif,1,Prospective,h/e,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Find all the paths

In [5]:
import os

root_dir = f"{working_directory}/{eval}/train/" + model

train_parquet_paths = []
val_parquet_paths = []
test_parquet_paths = []
all_parquets_paths = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                train_parquet_paths.append(file_path)
train_parquet_paths.sort()

#print("Sorted Parquet paths:")
#for path in train_parquet_paths:
#    print(path)

root_dir = f"{working_directory}/{eval}/val/" + model

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                val_parquet_paths.append(file_path)
val_parquet_paths.sort()


root_dir = f"{working_directory}/{eval}/test-int/" + model

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                test_parquet_paths.append(file_path)
test_parquet_paths.sort()

root_dir = f"{working_directory}/{eval}/all/int_folds_{model}"

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                all_parquets_paths.append(file_path)
all_parquets_paths.sort()


In [6]:
print(all_parquets_paths)

['/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00000-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00001-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00002-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00003-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00004-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00005-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00006-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00007-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00008-attention_mil/predictions.parquet', '/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00009-attention_mil/p

# Train Dummy models

In [7]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, accuracy_score

In [8]:
print(all_parquets_paths[0])

/media/mmd/Samsung_T5/GitHub/UMD/eval/all/int_folds_pdl1-3/00000-attention_mil/predictions.parquet


In [9]:
metrics_list = []
numbers = []

test_scores = []
test_true_y = []
test_pred_y =[]

test_list_dataframes = []




path = all_parquets_paths[0]

df_predictions = pd.read_parquet(path)
df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
#print(df_joined)
distribution = df_joined[["slide", outcome]]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_list = np.ones((distribution.shape[0], 1))
y_list = df_joined[outcome].to_numpy()

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_list, y_list)

y_pred = dummy_clf.predict(x_list)



df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
#print(df_joined)
distribution = df_joined[["slide",outcome]]

display(distribution)

y_true = df_joined[outcome].to_numpy()
print(y_true)
x_list = np.ones((distribution.shape[0], 1))



#print(distribution)
y_pred = dummy_clf.predict(x_list)
print(y_pred)

test_true_y.append(y_true)
test_pred_y.append(y_pred)

distribution["y_pred"] = y_pred
test_list_dataframes.append(distribution)




,slide,PDL1_CATHEGORY
0,DIG_PAT_1696325152,>=50 %
1,DIG_PAT_1696325261,1-49 %
2,DIG_PAT_1696325298,1-49 %
3,DIG_PAT_1696325346,1-49 %
4,DIG_PAT_1696325458,>=50 %
...,...,...
273,DIG_PAT_1717056286,1-49 %
274,DIG_PAT_1717056391,>=50 %
275,DIG_PAT_1717056457,1-49 %
276,DIG_PAT_1717056563,< 1 %


['>=50 %' '1-49 %' '1-49 %' '1-49 %' '>=50 %' '>=50 %' '< 1 %' '>=50 %'
 '1-49 %' '>=50 %' '< 1 %' '1-49 %' '1-49 %' '1-49 %' '< 1 %' '>=50 %'
 '>=50 %' '>=50 %' '1-49 %' '1-49 %' '< 1 %' '>=50 %' '1-49 %' '1-49 %'
 '1-49 %' '< 1 %' '>=50 %' '>=50 %' '< 1 %' '< 1 %' '< 1 %' '>=50 %'
 '< 1 %' '>=50 %' '>=50 %' '1-49 %' '>=50 %' '>=50 %' '1-49 %' '< 1 %'
 '< 1 %' '< 1 %' '1-49 %' '< 1 %' '1-49 %' '1-49 %' '< 1 %' '< 1 %'
 '< 1 %' '< 1 %' '1-49 %' '< 1 %' '1-49 %' '1-49 %' '1-49 %' '< 1 %'
 '1-49 %' '>=50 %' '< 1 %' '< 1 %' '1-49 %' '< 1 %' '1-49 %' '< 1 %'
 '< 1 %' '1-49 %' '>=50 %' '< 1 %' '>=50 %' '< 1 %' '< 1 %' '>=50 %'
 '1-49 %' '>=50 %' '1-49 %' '>=50 %' '1-49 %' '1-49 %' '1-49 %' '>=50 %'
 '>=50 %' '1-49 %' '>=50 %' '1-49 %' '1-49 %' '>=50 %' '1-49 %' '< 1 %'
 '>=50 %' '1-49 %' '>=50 %' '1-49 %' '< 1 %' '< 1 %' '< 1 %' '1-49 %'
 '< 1 %' '< 1 %' '1-49 %' '1-49 %' '1-49 %' '< 1 %' '1-49 %' '1-49 %'
 '1-49 %' '< 1 %' '1-49 %' '< 1 %' '1-49 %' '1-49 %' '1-49 %' '< 1 %'
 '1-49 %' '< 1 

/tmp/ipykernel_148818/1853329447.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  distribution["y_pred"] = y_pred


In [10]:
print(len(test_list_dataframes))

1


In [11]:
test_df = pd.concat(test_list_dataframes, ignore_index = True)
test_df.rename(columns={test_df.columns[1]: 'y_true'}, inplace=True)

In [12]:
display(test_df)

,slide,y_true,y_pred
0,DIG_PAT_1696325152,>=50 %,1-49 %
1,DIG_PAT_1696325261,1-49 %,1-49 %
2,DIG_PAT_1696325298,1-49 %,1-49 %
3,DIG_PAT_1696325346,1-49 %,1-49 %
4,DIG_PAT_1696325458,>=50 %,1-49 %
...,...,...,...
273,DIG_PAT_1717056286,1-49 %,1-49 %
274,DIG_PAT_1717056391,>=50 %,1-49 %
275,DIG_PAT_1717056457,1-49 %,1-49 %
276,DIG_PAT_1717056563,< 1 %,1-49 %


In [13]:
distribution

,slide,PDL1_CATHEGORY,y_pred
0,DIG_PAT_1696325152,>=50 %,1-49 %
1,DIG_PAT_1696325261,1-49 %,1-49 %
2,DIG_PAT_1696325298,1-49 %,1-49 %
3,DIG_PAT_1696325346,1-49 %,1-49 %
4,DIG_PAT_1696325458,>=50 %,1-49 %
...,...,...,...
273,DIG_PAT_1717056286,1-49 %,1-49 %
274,DIG_PAT_1717056391,>=50 %,1-49 %
275,DIG_PAT_1717056457,1-49 %,1-49 %
276,DIG_PAT_1717056563,< 1 %,1-49 %


In [14]:
y_true

array(['>=50 %', '1-49 %', '1-49 %', '1-49 %', '>=50 %', '>=50 %',
       '< 1 %', '>=50 %', '1-49 %', '>=50 %', '< 1 %', '1-49 %', '1-49 %',
       '1-49 %', '< 1 %', '>=50 %', '>=50 %', '>=50 %', '1-49 %',
       '1-49 %', '< 1 %', '>=50 %', '1-49 %', '1-49 %', '1-49 %', '< 1 %',
       '>=50 %', '>=50 %', '< 1 %', '< 1 %', '< 1 %', '>=50 %', '< 1 %',
       '>=50 %', '>=50 %', '1-49 %', '>=50 %', '>=50 %', '1-49 %',
       '< 1 %', '< 1 %', '< 1 %', '1-49 %', '< 1 %', '1-49 %', '1-49 %',
       '< 1 %', '< 1 %', '< 1 %', '< 1 %', '1-49 %', '< 1 %', '1-49 %',
       '1-49 %', '1-49 %', '< 1 %', '1-49 %', '>=50 %', '< 1 %', '< 1 %',
       '1-49 %', '< 1 %', '1-49 %', '< 1 %', '< 1 %', '1-49 %', '>=50 %',
       '< 1 %', '>=50 %', '< 1 %', '< 1 %', '>=50 %', '1-49 %', '>=50 %',
       '1-49 %', '>=50 %', '1-49 %', '1-49 %', '1-49 %', '>=50 %',
       '>=50 %', '1-49 %', '>=50 %', '1-49 %', '1-49 %', '>=50 %',
       '1-49 %', '< 1 %', '>=50 %', '1-49 %', '>=50 %', '1-49 %', '< 1 %',
 

In [15]:
def extract_5_elements (train_true_y, train_pred_y):
    liste =train_true_y
    y_trues = []
    for i in range(0, len(liste), 5):
        unione = [item for sublist in liste[i:i+5] for item in sublist]
        y_trues.append(unione)
        #print(unione)

    liste = train_pred_y
    y_preds = []
    for i in range(0, len(liste), 5):
        unione = [item for sublist in liste[i:i+5] for item in sublist]
        y_preds.append(unione)
        #print(unione)
    return y_trues, y_preds


In [16]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

In [17]:
def average(df_input, name): 
    df = df_input.copy()
    # Removing "Fold" column
    df = df.drop(columns=["Dataset","Fold"])  
    mean_values = df.mean()
    std_values = df.std()
    n = len(df)
    ci_values = (2 * std_values) / np.sqrt(n)
    result_df = pd.DataFrame({
    "Column": mean_values.index,
    "Mean": mean_values.values,
    "CI": ci_values.values
    })
    result_df = result_df.add_prefix(name)
    display(result_df)
    return result_df

In [18]:
def extract_metrics(train_true_y, train_pred_y, model, name_set):
    y_trues, y_preds = extract_5_elements(train_true_y, train_pred_y)
    results = []  # Store results

    for i in range(len(y_trues)):
        y_list = y_trues[i]
        y_pred = y_preds[i]
        acc = accuracy_score(y_list, y_pred)
        precision = precision_score(y_list, y_pred, average="weighted", zero_division=0)
        recall = recall_score(y_list, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_list, y_pred, average="weighted", zero_division=0)

        report = classification_report(y_list, y_pred, digits=2, output_dict=True, zero_division=0)

        # Flatten report into a list
        row = {"Dataset": name_set, "Fold": i+1, "Accuracy": acc, "Precision" : precision, "Recall": recall, "F1_score" : f1}

        for label, metrics in report.items():
            if isinstance(metrics, dict):  # Avoid 'accuracy' which is a single value
                for metric_name, value in metrics.items():
                    row[f"{label}_{metric_name}"] = value
        
        results.append(row)

    # Convert to DataFrame
    df_results = pd.DataFrame(results)


    display(df_results)
    #df_results.to_excel(model+f"_{name_set}_5_folds.xlsx")

    avg = average(df_results, name_set)

    return df_results, avg
    


In [19]:
#df1, avg_1 = extract_metrics( train_true_y, train_pred_y, model, "train")
#df2, avg_2= extract_metrics( val_true_y, val_pred_y, model, "val")
df3, avg_3 = extract_metrics( test_true_y, test_pred_y, model, "test")

#df_final = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
#df_final.to_excel(model+f"_complete_5_folds.xlsx")

#df_final_avg = pd.concat([avg_1, avg_2, avg_3], axis=1, ignore_index=False)
#df_final_avg.to_excel(model+f"_complete_averages_5_folds.xlsx")

,Dataset,Fold,Accuracy,Precision,Recall,F1_score,1-49 %_precision,1-49 %_recall,1-49 %_f1-score,1-49 %_support,...,>=50 %_f1-score,>=50 %_support,macro avg_precision,macro avg_recall,macro avg_f1-score,macro avg_support,weighted avg_precision,weighted avg_recall,weighted avg_f1-score,weighted avg_support
0,test,1,0.356115,0.126818,0.356115,0.187031,0.356115,1.0,0.525199,99.0,...,0.0,81.0,0.118705,0.333333,0.175066,278.0,0.126818,0.356115,0.187031,278.0


,testColumn,testMean,testCI
0,Accuracy,0.356115,NaN
1,Precision,0.126818,NaN
2,Recall,0.356115,NaN
3,F1_score,0.187031,NaN
4,1-49 %_precision,0.356115,NaN
5,1-49 %_recall,1.000000,NaN
6,1-49 %_f1-score,0.525199,NaN
7,1-49 %_support,99.000000,NaN
8,< 1 %_precision,0.000000,NaN
9,< 1 %_recall,0.000000,NaN


In [20]:
from unittest import TestCase
test_df

,slide,y_true,y_pred
0,DIG_PAT_1696325152,>=50 %,1-49 %
1,DIG_PAT_1696325261,1-49 %,1-49 %
2,DIG_PAT_1696325298,1-49 %,1-49 %
3,DIG_PAT_1696325346,1-49 %,1-49 %
4,DIG_PAT_1696325458,>=50 %,1-49 %
...,...,...,...
273,DIG_PAT_1717056286,1-49 %,1-49 %
274,DIG_PAT_1717056391,>=50 %,1-49 %
275,DIG_PAT_1717056457,1-49 %,1-49 %
276,DIG_PAT_1717056563,< 1 %,1-49 %


In [21]:
# Raggruppiamo i dati per slide e y_true e creiamo nuove colonne per ogni valore di y_pred_max_idx
df_final = test_df.groupby(["slide", "y_true"])["y_pred"].agg(list).reset_index()

# Espandiamo la lista in colonne separate
df_final_expanded = df_final["y_pred"].apply(pd.Series)

# Rinominiamo le nuove colonne
df_final_expanded.columns = [f"pred_{i+1}" for i in df_final_expanded.columns]

# Concatenazione con le colonne originali
df_result = pd.concat([df_final.drop(columns=["y_pred"]), df_final_expanded], axis=1)

In [22]:
df_result

,slide,y_true,pred_1
0,DIG_PAT_1696325152,>=50 %,1-49 %
1,DIG_PAT_1696325261,1-49 %,1-49 %
2,DIG_PAT_1696325298,1-49 %,1-49 %
3,DIG_PAT_1696325346,1-49 %,1-49 %
4,DIG_PAT_1696325458,>=50 %,1-49 %
...,...,...,...
273,DIG_PAT_1717056286,1-49 %,1-49 %
274,DIG_PAT_1717056391,>=50 %,1-49 %
275,DIG_PAT_1717056457,1-49 %,1-49 %
276,DIG_PAT_1717056563,< 1 %,1-49 %


In [23]:
a = [col for col in df_result.columns if 'pred_' in col]

In [24]:

# Supponiamo che il tuo DataFrame si chiami df
colonne_pred = [col for col in df_result.columns if 'pred_' in col]#[f"pred_{i}" for i in range(1, 6)]

# Calcola il valore più frequente (majority vote) per ogni riga
df_result['y_pred'] = df_result[colonne_pred].mode(axis=1)[0]

# Visualizza il DataFrame aggiornato
print(df_result.head())

                slide  y_true  pred_1  y_pred
0  DIG_PAT_1696325152  >=50 %  1-49 %  1-49 %
1  DIG_PAT_1696325261  1-49 %  1-49 %  1-49 %
2  DIG_PAT_1696325298  1-49 %  1-49 %  1-49 %
3  DIG_PAT_1696325346  1-49 %  1-49 %  1-49 %
4  DIG_PAT_1696325458  >=50 %  1-49 %  1-49 %


In [25]:
df_test = df_result[["slide", "y_true", "y_pred"]]

In [26]:
df_test

,slide,y_true,y_pred
0,DIG_PAT_1696325152,>=50 %,1-49 %
1,DIG_PAT_1696325261,1-49 %,1-49 %
2,DIG_PAT_1696325298,1-49 %,1-49 %
3,DIG_PAT_1696325346,1-49 %,1-49 %
4,DIG_PAT_1696325458,>=50 %,1-49 %
...,...,...,...
273,DIG_PAT_1717056286,1-49 %,1-49 %
274,DIG_PAT_1717056391,>=50 %,1-49 %
275,DIG_PAT_1717056457,1-49 %,1-49 %
276,DIG_PAT_1717056563,< 1 %,1-49 %


In [27]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
def extract_metrics(train_true_y, train_pred_y, model, name_set):
    
    results = []  # Store results
    y_list = train_true_y
    y_pred = train_pred_y
    acc = accuracy_score(y_list, y_pred)

    precision = precision_score(y_list, y_pred, average="weighted", zero_division=0)
    weighted_accuracy = balanced_accuracy_score(y_list, y_pred)

    recall = recall_score(y_list, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_list, y_pred, average="weighted", zero_division=0)

    report = classification_report(y_list, y_pred, digits=2,   output_dict=True, zero_division=0)

    # Flatten report into a list
    row = {"Dataset": name_set, "Fold": 1, "Accuracy": acc, "Balanced Accuracy":weighted_accuracy, "Precision" : precision, "Recall": recall, "F1_score" : f1}

    for label, metrics in report.items():
        if isinstance(metrics, dict):  # Avoid 'accuracy' which is a single value
            for metric_name, value in metrics.items():
                row[f"{label}_{metric_name}"] = value
        
    results.append(row)

    # Convert to DataFrame
    df_results = pd.DataFrame(results)


    display(df_results)
    #df_results.to_excel(model+f"_{name_set}_5_folds.xlsx")

    

    return df_results

In [28]:
y_true = df_test["y_true"].to_numpy()
y_pred = df_test["y_pred"].to_numpy()

In [29]:
df = extract_metrics(y_true, y_pred, "br3", "test")

,Dataset,Fold,Accuracy,Balanced Accuracy,Precision,Recall,F1_score,1-49 %_precision,1-49 %_recall,1-49 %_f1-score,...,>=50 %_f1-score,>=50 %_support,macro avg_precision,macro avg_recall,macro avg_f1-score,macro avg_support,weighted avg_precision,weighted avg_recall,weighted avg_f1-score,weighted avg_support
0,test,1,0.356115,0.333333,0.126818,0.356115,0.187031,0.356115,1.0,0.525199,...,0.0,81.0,0.118705,0.333333,0.175066,278.0,0.126818,0.356115,0.187031,278.0
